# 垃圾分拣播报玩法

### 导入头文件

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import Arm_Lib
import cv2 as cv
import threading
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from speech_garbage_identify import speech_garbage_identify

### 创建实例,初始化参数

In [ ]:
# 创建获取目标实例
single_garbage = speech_garbage_identify()
# 初始化模式
model = "General"

### 初始化机械臂位置

In [ ]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [90, 90, 20, 15, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件

In [ ]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

### 退出模式控件

In [ ]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
exit_button.on_click(exit_button_Callback)

### 主程序

In [ ]:
def camera():
    # 打开摄像头
    capture = cv.VideoCapture(0)
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            # 读取相机的每一帧
            _, img = capture.read()
            # 统一图像大小
            img = cv.resize(img, (640, 480))
            img = single_garbage.single_garbage_run(img)
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()